In [1]:
import os
import sys
import pandas as pd

# 添加项目根目录到 Python 路径
project_root = '/date0/crwu/zjts_foodname'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from packages.file_deal import read_dataset

2025-10-14 09:39:54 - file_deal - INFO - [my_logger.py:103] - 日志系统初始化完成，日志文件: /date0/crwu/zjts_foodname/logs/20251014_093954_file_deal.log
2025-10-14 09:39:54 - file_deal - INFO - [my_logger.py:104] - 脚本名称: file_deal


# 读各个数据源，包括分类数据库， 食物分类层级， 食物标签

In [ ]:
from collections import defaultdict

csv_root = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/'
csv_data = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C10/merged_results.csv'
food_depth_path = r'/date0/crwu/zjts_foodname/data/classification/data/food_depth.csv'
food_tags_path = r'/date0/crwu/zjts_foodname/data/classification/data/food_tags.csv'

depth_df = read_dataset(food_depth_path)
tags_df = read_dataset(food_tags_path)
df = read_dataset(csv_data)
rule_dict = defaultdict(set)
for row in depth_df.itertuples():
    _, *levels = tuple(row)
    rule_dict['第一级别'].add(','.join(levels))
    rule_dict['第二级别'].add(','.join(levels[1:]))
    rule_dict['第三级别'].add(','.join(levels[2:]))
    rule_dict['第四级别'].add(','.join(levels[3:]))

    rule_dict['第一级别'].add(levels[0])
    rule_dict['第二级别'].add(levels[1])
    rule_dict['第三级别'].add(levels[2])
    rule_dict['第四级别'].add(levels[3])

for column in tags_df.columns:
    for val in tags_df[column]:
        if isinstance(val, float) and not pd.isna(val):
            val = str(int(val))
        if not pd.isna(val):
            rule_dict[column].add(val)
print(rule_dict) 



2025-10-14 09:39:58 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/data/food_depth.csv ...
2025-10-14 09:39:58 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/data/food_tags.csv ...
2025-10-14 09:39:58 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C10/merged_results.csv ...
defaultdict(<class 'set'>, {'第一级别': {'H. 饮品,H. 饮品,酒精饮品,黄酒/米酒', 'F. 汤羹类,F. 汤羹类,药膳/滋补汤,药膳/滋补汤', 'C. 以蔬菜/水果为主的菜品,C1. 蔬菜类,浅色叶菜,浅色叶菜', 'A. 富含优质蛋白质的菜品,A1. 红肉类,牛肉,牛肉丝', 'B. 以碳水化合物食材为主的菜品,B1. 精制谷物类,精面粉面食,油条', 'A. 富含优质蛋白质的菜品,A6. 蛋制品类,未处理的蛋,鸡蛋', 'A. 富含优质蛋白质的菜品,A8. 植物蛋白类,豆制品,腐竹/豆皮', 'F. 汤羹类,F. 汤羹类,浓汤/奶汤,浓汤/奶汤', 'A. 富含优质蛋白质的菜品,A6. 蛋制品类,未处理的蛋,鸭蛋', 'A. 富含优质蛋白质的菜品,A1. 红肉类,羊肉,羊蝎子', 'B. 以碳水化合物食材为主的菜品,B1. 精制谷物类,精米饭/米粉/米制品,炒饭', 'G. 甜品,G. 甜品,中式甜汤,中式甜汤', 'A. 富含优质蛋白质的菜品,A2. 禽肉类,其他禽肉,其他禽肉', 'A. 富含优质蛋白质的菜品

# 合并表格用的

In [ ]:
# csv_data_2 = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/merged_results.csv'

# depth_df = read_dataset(food_depth_path)
# tags_df = read_dataset(food_tags_path)
# df_2 = read_dataset(csv_data_2)
# df['营养建议分类'] = df_2['营养建议分类']
# df['早中晚餐消费场景'] = df_2['早中晚餐消费场景']
# df['食用频率'] = df_2['食用频率']
# df['熟悉程度'] = df_2['熟悉程度']
# print(df.iloc[0:10,0:15])
# df.to_csv(os.path.join(csv_root, 'food_name_database_V100R25C20.csv'), index=False)

In [3]:
failed_vals_dict = {}
for column in df.columns:
    if column == '菜品名':
        continue    
    if column == '主要食材种类' or column == '营养建议分类':
        val_list = df[column]
        valid_values = set()
        for vals in val_list:
            for val in vals.split('/'):
                valid_values.add(val)
    else:
        valid_values = set(df[column])
    for val in list(valid_values):
        if (val.startswith('(') and val.endswith(')')) or val == '不适用':
            valid_values.remove(val)
    print(f'{column}共有{len(valid_values)}个取值')
    not_qulified_vals = valid_values - rule_dict[column]
    print(f'不在rule中的值有{len(not_qulified_vals)}个: {not_qulified_vals}')
    if not_qulified_vals:    
        failed_vals_dict[column] = list(not_qulified_vals)
print(failed_vals_dict)

第一级别共有7个取值
不在rule中的值有0个: set()
第二级别共有17个取值
不在rule中的值有0个: set()
第三级别共有52个取值
不在rule中的值有0个: set()
第四级别共有155个取值
不在rule中的值有0个: set()
主要食材种类共有52个取值
不在rule中的值有2个: {'淡水鱼', '海水鱼'}
烹饪方式共有22个取值
不在rule中的值有0个: set()
菜系（地域）分类共有21个取值
不在rule中的值有0个: set()
消费场景共有6个取值
不在rule中的值有0个: set()
营养建议分类共有5个取值
不在rule中的值有0个: set()
记忆/推理菜共有2个取值
不在rule中的值有0个: set()
{'主要食材种类': ['淡水鱼', '海水鱼']}


# 淡水鱼和海水鱼是手动删掉合并的，切记切记

In [4]:

failed_row_index_set = set()
for column, failed_val_list in failed_vals_dict.items():
    if column != '主要食材种类':
        failed_df = df[df[column].isin(failed_val_list)]
        print(f'打印"{column}"的失败项,共计{len(failed_df)}个')
        print(failed_df)
        print('*' * 50)
        for index in failed_df.index:
            failed_row_index_set.add(index)
    else:
        count = 0
        for row in df.iterrows():
            # print()
            # break
            for val in failed_val_list:
                if val in row[1].iloc[5].split('/'):
                    count += 1
                    print(row[1])
                    failed_row_index_set.add(row[0])
                    break
        print(f'打印"{column}"的失败项,共计{count}个')
        # print(failed_df)
        print('*' * 50)


菜品名                     三文鱼粥
第一级别        B. 以碳水化合物食材为主的菜品
第二级别               B1. 精制谷物类
第三级别              精米饭/米粉/米制品
第四级别                       粥
主要食材种类                米类/海水鱼
烹饪方式                       煮
菜系（地域）分类            家常/不区分菜系
消费场景                     家常菜
营养建议分类          主食/淀粉类/高糖/高油
记忆/推理菜                   推理菜
Name: 70, dtype: object
菜品名                      带鱼粥
第一级别        B. 以碳水化合物食材为主的菜品
第二级别               B1. 精制谷物类
第三级别              精米饭/米粉/米制品
第四级别                       粥
主要食材种类                米类/海水鱼
烹饪方式                       煮
菜系（地域）分类            家常/不区分菜系
消费场景                     家常菜
营养建议分类          主食/淀粉类/高糖/高油
记忆/推理菜                   推理菜
Name: 148, dtype: object
菜品名                    柴鱼花生粥
第一级别        B. 以碳水化合物食材为主的菜品
第二级别               B1. 精制谷物类
第三级别              精米饭/米粉/米制品
第四级别                       粥
主要食材种类         米类/海水鱼/坚果或种子类
烹饪方式                       煮
菜系（地域）分类            家常/不区分菜系
消费场景                     家常菜
营养建议分类          主食/淀粉类/高糖/高油
记忆/推理菜                 

In [5]:
fail_count = 0
for row in df.iterrows():
    lv1, lv2, lv3, lv4 = tuple(row[1].iloc[1:5])
    if ",".join(list(row[1].iloc[1:5])) not in rule_dict["第一级别"] and \
    lv1 != '(非菜品)' and \
    lv1 != '不适用' and \
    lv2 != '不适用' and \
    lv3 != '不适用' and \
    lv4 != '不适用' and \
        True:
        fail_count += 1
        print(list(row[1].iloc[0:5]))
        failed_row_index_set.add(row[0])
print(f'{fail_count}条数据层级不对')

0条数据层级不对


In [6]:
print(failed_row_index_set)
print(f'失败行共{len(failed_row_index_set)}个')
failed_files_set = set()
for failed_row_index in list(failed_row_index_set):
    failed_files_set.add(failed_row_index//5)
print(failed_files_set)
print(f'失败文件共{len(failed_files_set)}个')

    

{24590, 70, 16502, 148, 175, 253, 259, 16666, 16667, 8496, 370, 383, 468, 493, 494, 495, 504, 517, 527, 22965, 22967, 22968, 22969, 22970, 920, 936, 937, 25556, 25578, 25579, 25580, 25581, 25583, 25584, 25585, 25588, 25589, 25590, 25591, 25593, 25595, 25596, 25598, 25600, 25601, 25602, 25608, 25610, 25611, 25612, 25613, 25614, 25615, 25616, 25617, 25619, 25620, 25621, 25624, 25626, 25627, 25628, 25629, 25630, 25631, 25633, 25634, 25637, 25639, 25640, 25641, 25643, 25644, 25645, 25646, 25647, 25648, 25649, 25650, 25651, 25652, 25654, 25657, 25658, 25659, 25661, 25663, 25665, 25666, 25667, 25668, 25669, 25670, 25673, 25674, 25675, 25676, 25680, 25720, 25721, 25723, 25729, 25731, 25739, 25744, 25759, 25781, 17596, 25792, 25793, 25794, 25795, 25797, 25798, 25799, 25800, 25801, 25802, 25805, 17614, 25806, 25807, 17620, 1240, 17638, 25835, 25836, 17646, 17647, 17649, 17664, 25857, 25882, 25912, 25917, 25926, 25989, 25990, 17851, 1495, 26084, 26089, 26096, 26117, 26132, 26134, 1560, 1586, 179

# 删除有问题的.pk文件然后重跑打标签代码，对有问题的部分重新打标

In [221]:
import os
pk_file_root = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C10'
for failed_pk_index in list(failed_files_set):
    try:        
        # os.remove(os.path.join(pk_file_root, f'{failed_pk_index}.pk'))
        print(f'删除失败文件{failed_pk_index}.pk')
    except:
        pass